Need to crate conda environment

1. conda create -n xclim_env
2. conda activate xclim_env
3. conda install xclim
4. conda install h5netcdf

In [4]:
import xclim
from xclim.testing import utils
import xarray as xr

In [7]:
ds = xr.open_dataset("daily_surface_cancities_1990-1993.nc", engine="h5netcdf")

ds = ds.assign(
     hurs=xclim.atmos.relative_humidity_from_dewpoint(ds=ds),
     tas=xclim.core.units.convert_units_to(ds.tas, "degC"),
     pr=xclim.core.units.convert_units_to(ds.pr, "mm/d"),
     sfcWind=xclim.atmos.wind_speed_from_vector(ds=ds)[0],
)

season_mask = xclim.atmos.fire_season(
    tas=ds.tas,
    method="WF93",
    freq="YS",
     # Parameters below are at their default values, but listed here for explicitness.
    temp_start_thresh="12 degC",
    temp_end_thresh="5 degC",
    temp_condition_days=3,
)

output = xclim.indices.cffwis_indices(
     tas=ds.tas,
     pr=ds.pr,
     hurs=ds.hurs,
     sfcWind=ds.sfcWind,
     lat=ds.lat,
     season_mask=season_mask,
     overwintering=True,
     dry_start="CFS",
     prec_thresh="1.5 mm/d",
     dmc_dry_factor=1.2,
     # Parameters below are at their default values, but listed here for explicitness.
     carry_over_fraction=0.75,
     wetting_efficiency_fraction=0.75,
     dc_start=15,
     dmc_start=6,
     ffmc_start=85,
)

fwi = output[-1]

/Users/ericcollins/opt/miniconda3/envs/xclim/lib/python3.12/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable does not have a `cell_methods` attribute.
  _check_cell_methods(
/Users/ericcollins/opt/miniconda3/envs/xclim/lib/python3.12/site-packages/xclim/core/cfchecks.py:47: UserWarning: Variable does not have a `standard_name` attribute.
  check_valid(vardata, "standard_name", data["standard_name"])


<xarray.DataArray (location: 5, time: 1461)> Size: 7kB
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
Coordinates:
    lon       (location) float32 20B -63.4 -73.4 -68.4 -106.7 -123.2
  * location  (location) <U9 180B 'Halifax' 'Montréal' ... 'Victoria'
    lat       (location) float32 20B 44.5 45.5 63.75 52.0 48.5
  * time      (time) datetime64[ns] 12kB 1990-01-01 1990-01-02 ... 1993-12-31